In [31]:
import polars as pl

pl.Config.set_tbl_rows(50)
pl.Config.set_tbl_cols(-1)

polars.config.Config

In [32]:
tracking_df = pl.read_parquet("../data/split_prepped_data/*_features.parquet")
play_df = pl.read_csv("../data/bdb_2024/plays.csv", null_values=["", "NA", "na", "nan", "NaN", "NAN"]).with_columns(
    distanceToGoal=(
        pl.when(pl.col("possessionTeam") == pl.col("yardlineSide"))
        .then(100 - pl.col("yardlineNumber"))
        .otherwise(pl.col("yardlineNumber"))
    )
)

play_df.sample(3)

gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,distanceToGoal
i64,i64,i64,str,str,i64,i64,i64,str,str,str,i64,str,i64,i64,str,i64,i64,i64,i64,str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,str,str,i64,str,i64
2022091811,915,47932,"""Hunter Renfrow""","""(13:05) (Shotgun) D.Carr pass …",2,1,20,"""LV""","""ARI""","""ARI""",29,"""13:05""",7,0,"""C""",-3,null,8,8,"""N""",39,"""SHOTGUN""",6,0.716908,0.896502,0.103498,-0.027158,0.027158,3.789393,0.057752,null,null,null,null,29
2022091107,3551,53463,"""Elijah Moore""","""(5:17) (Shotgun) J.Flacco pass…",4,4,3,"""NYJ""","""BAL""","""BAL""",14,"""5:17""",3,24,"""C""",4,null,4,4,"""N""",96,"""SHOTGUN""",5,0.960585,0.003442,0.996558,0.00131,-0.00131,2.877711,2.36316,null,null,null,null,14
2022102306,3352,44852,"""Curtis Samuel""","""(10:10) (Shotgun) T.Heinicke p…",4,3,8,"""WAS""","""GB""","""WAS""",39,"""10:10""",20,14,"""C""",4,15,26,41,"""N""",71,"""SHOTGUN""",5,0.979484,0.781938,0.218062,0.103929,-0.103929,0.680333,3.899961,"""Unsportsmanlike Conduct""",null,53458,null,61


In [33]:
results_df = (
    pl.concat(
        [
            pl.read_parquet("../data/best_models/zoo/best_model_results.parquet"),
            pl.read_parquet("../data/best_models/transformer/best_model_results.parquet"),
        ]
    )
    .join(
        tracking_df.filter(pl.col("is_ball_carrier") == 1)
        .select(["x", "y", "gameId", "playId", "frameId", "mirrored", "event"])
        .rename({"x": "ball_carrier_x", "y": "ball_carrier_y"}),
        on=["gameId", "playId", "frameId", "mirrored"],
        how="inner",
    )
    .with_columns(
        frame_difference_from_tackle=(pl.col("tackle_frameId") - pl.col("frameId")),
    )
    .with_columns(
        frame_difference_from_tackle_cat=pl.col("frame_difference_from_tackle").cut(
            range(-10, 51, 5), left_closed=True
        ),
    )
)
print(len(results_df))
results_df.sample(10)

2118428


gameId,playId,mirrored,ballCarrierNflId,ballCarrierName,tackle_frameId,tackle_event,tackle_x,tackle_y,tackle_x_rel,tackle_y_rel,play_origin_x,play_origin_y,playResult,tackle_event_enum,frameId,dataset_split,tackle_x_rel_pred,tackle_y_rel_pred,tackle_x_pred,tackle_y_pred,params,model_type,batch_size,hidden_dim,num_layers,advanced,dropout,learning_rate,ball_carrier_x,ball_carrier_y,event,frame_difference_from_tackle,frame_difference_from_tackle_cat
i64,i64,bool,i64,str,i64,str,f64,f64,f64,f64,f64,f64,i64,i64,i64,str,f32,f32,f64,f64,i32,str,i32,i32,i32,bool,f64,f64,f64,f64,str,i64,cat
2022091809,1407,true,43293,"""Ezekiel Elliott""",46,"""tackle""",37.74,20.1,10.13,-5.3,27.61,25.4,2,0,10,"""train""",8.8,-7.7,36.5,17.7,55990,"""zoo""",256,128,2,false,0.3,0.0001,27.78,25.22,null,36,"""[35, 40)"""
2022102309,3809,true,54506,"""Kenneth Walker""",123,"""touchdown""",111.05,44.78,82.38,21.37,28.67,23.41,74,2,25,"""val""",6.9,21.4,35.5,44.8,12680078,"""transformer""",256,512,4,false,0.3,0.0001,29.97,31.23,null,98,"""[50, inf)"""
2022110606,402,true,46076,"""Josh Allen""",62,"""tackle""",84.71,15.19,11.59,-14.21,73.12,29.4,7,0,33,"""train""",11.6,-6.8,84.7,22.6,55990,"""zoo""",256,128,2,false,0.3,0.0001,78.05,27.01,null,29,"""[25, 30)"""
2022102303,1919,true,43293,"""Ezekiel Elliott""",66,"""tackle""",56.62,48.32,24.61,18.67,32.01,29.65,18,0,21,"""train""",14.9,1.9,46.9,31.6,12680078,"""transformer""",256,512,4,false,0.3,0.0001,36.22,30.87,null,45,"""[45, 50)"""
2022091111,770,false,41282,"""Davante Adams""",9,"""out_of_bounds""",17.73,0.4,-0.48,-5.86,18.21,6.26,5,1,10,"""train""",0.2,-8.9,18.4,-2.6,55990,"""zoo""",256,128,2,false,0.3,0.0001,17.71,0.13,null,-1,"""[-5, 0)"""
2022092504,2795,false,44947,"""Jamaal Williams""",64,"""touchdown""",109.43,1.55,20.46,-25.22,88.97,26.77,13,2,10,"""train""",11.6,-4.7,100.5,22.1,12680078,"""transformer""",256,512,4,false,0.3,0.0001,89.1,26.69,null,54,"""[50, inf)"""
2022102309,2686,true,54506,"""Kenneth Walker""",57,"""tackle""",63.59,11.93,15.9,-11.52,47.69,23.45,8,0,41,"""train""",14.7,-11.8,62.4,11.7,12680078,"""transformer""",256,512,4,false,0.3,0.0001,60.33,14.17,null,16,"""[15, 20)"""
2022100908,2582,false,54556,"""Rachaad White""",54,"""tackle""",69.07,25.71,7.85,-3.95,61.22,29.66,2,0,20,"""train""",8.1,-3.4,69.3,26.3,12680078,"""transformer""",256,512,4,false,0.3,0.0001,63.93,27.97,null,34,"""[30, 35)"""
2022091106,466,true,46203,"""Chase Edmonds""",43,"""tackle""",92.83,24.39,10.59,3.16,82.24,21.23,5,0,21,"""test""",9.3,4.3,91.5,25.6,55990,"""zoo""",256,128,2,false,0.3,0.0001,84.56,24.0,null,22,"""[20, 25)"""


In [34]:
# from sklearn.metrics import mean_squared_error
# from torch.nn.functional import mse_loss
# from torch import tensor
import numpy as np


def calculate_mse(x: pl.Series, y: pl.Series, xhat: pl.Series, yhat: pl.Series):
    """
    Calculate the mean squared error between the predicted and true values of x and y.
    """
    x, y, xhat, yhat = x.to_numpy(), y.to_numpy(), xhat.to_numpy(), yhat.to_numpy()
    return np.mean((np.array([xhat - x, yhat - y]) ** 2).mean(axis=0))


def calculate_mae(x: pl.Series, y: pl.Series, xhat: pl.Series, yhat: pl.Series):
    """
    Calculate the mean average error between the predicted and true values of x and y.
    """
    x, y, xhat, yhat = x.to_numpy(), y.to_numpy(), xhat.to_numpy(), yhat.to_numpy()
    return np.mean(np.abs(np.array([xhat - x, yhat - y])).mean(axis=0))


(
    results_df.group_by(["dataset_split", "model_type"], maintain_order=True)
    .agg(
        n_frames=pl.struct(["gameId", "playId", "frameId", "mirrored"]).n_unique(),
        n_plays=pl.struct(["gameId", "playId"]).n_unique(),
        score=pl.map_groups(
            exprs=["tackle_x", "tackle_y", "tackle_x_pred", "tackle_y_pred"],
            function=lambda list_of_series: calculate_mse(*list_of_series),
            # function=lambda list_of_series: calculate_mae(*list_of_series),
            returns_scalar=True,
        ).round(1),
    )
    .pivot(values="score", columns=["model_type"], index=["dataset_split", "n_plays", "n_frames"])
    .with_columns(perf_perc_diff=((pl.col("zoo") - pl.col("transformer")) * 100 / pl.col("zoo")).round(1))
)

dataset_split,n_plays,n_frames,zoo,transformer,perf_perc_diff
str,u32,u32,f64,f64,f64
"""train""",8735,740590,42.4,33.3,21.5
"""val""",1872,158500,45.8,36.7,19.9
"""test""",1871,160124,28.4,23.9,15.8


In [35]:

import polars as pl

df = pl.DataFrame(
    {
        "depth": [1, 3, 1, 2, 3],
        "value_depth1": [2, 15, 51, 12, 1],
        "value_depth2": [4, 9, 5, 1, 2],
        "value_depth3": [11, 0, 5, 7, 6],
        "value": [2, 0, 51, 1, 6],
    }
)
print(df)

shape: (5, 5)
┌───────┬──────────────┬──────────────┬──────────────┬───────┐
│ depth ┆ value_depth1 ┆ value_depth2 ┆ value_depth3 ┆ value │
│ ---   ┆ ---          ┆ ---          ┆ ---          ┆ ---   │
│ i64   ┆ i64          ┆ i64          ┆ i64          ┆ i64   │
╞═══════╪══════════════╪══════════════╪══════════════╪═══════╡
│ 1     ┆ 2            ┆ 4            ┆ 11           ┆ 2     │
│ 3     ┆ 15           ┆ 9            ┆ 0            ┆ 0     │
│ 1     ┆ 51           ┆ 5            ┆ 5            ┆ 51    │
│ 2     ┆ 12           ┆ 1            ┆ 7            ┆ 1     │
│ 3     ┆ 1            ┆ 2            ┆ 6            ┆ 6     │
└───────┴──────────────┴──────────────┴──────────────┴───────┘


In [36]:
(
    results_df.filter(pl.col("dataset_split") == "test")
    .group_by(["model_type", "frame_difference_from_tackle_cat"])
    .agg(
        n_frames=pl.len(),
        n_plays=pl.struct(["gameId", "playId"]).n_unique(),
        score=pl.map_groups(
            exprs=["tackle_x", "tackle_y", "tackle_x_pred", "tackle_y_pred"],
            function=lambda list_of_series: calculate_mse(*list_of_series),
            # function=lambda list_of_series: calculate_mae(*list_of_series),
            returns_scalar=True,
        ).round(1),
    )
    .sort("frame_difference_from_tackle_cat")
    .pivot(values="score", columns=["model_type"], index=["frame_difference_from_tackle_cat", "n_frames"])
    .with_columns(perf_perc_diff=((pl.col("zoo") - pl.col("transformer")) * 100 / pl.col("zoo")).round(1))
)

frame_difference_from_tackle_cat,n_frames,transformer,zoo,perf_perc_diff
cat,u32,f64,f64,f64
"""[-5, 0)""",14960,2.4,11.8,79.7
"""[0, 5)""",18710,2.1,9.2,77.2
"""[5, 10)""",18496,2.0,7.8,74.4
"""[10, 15)""",17358,2.7,7.4,63.5
"""[15, 20)""",15904,5.2,8.5,38.8
"""[20, 25)""",14456,9.2,11.4,19.3
"""[25, 30)""",12886,14.4,15.6,7.7
"""[30, 35)""",11536,19.8,20.8,4.8
"""[35, 40)""",10182,26.7,28.0,4.6


In [41]:
# split loss up by scramble, run, pass
(
    results_df.filter(pl.col("dataset_split").is_in(["test"]))
    .filter(pl.col("event").is_not_null())
    .group_by(["model_type", "event"])
    .agg(
        n_frames=pl.len(),
        n_plays=pl.struct(["gameId", "playId"]).n_unique(),
        avg_frameId=pl.col("frameId").mean().round(1),
        score=pl.map_groups(
            exprs=["tackle_x", "tackle_y", "tackle_x_pred", "tackle_y_pred"],
            function=lambda list_of_series: calculate_mse(*list_of_series),
            # function=lambda list_of_series: calculate_mae(*list_of_series),
            returns_scalar=True,
        ).round(1),
    )
    .sort(["event", "model_type"])
    .pivot(values="score", columns=["model_type"], index=["event", "n_plays", "n_frames", "avg_frameId"])
    .filter(pl.col("n_plays") > 100)
    .with_columns(perf_perc_diff=((pl.col("zoo") - pl.col("transformer")) * 100 / pl.col("zoo")).round(1))
    .sort("avg_frameId")
)

event,n_plays,n_frames,avg_frameId,transformer,zoo,perf_perc_diff
str,u32,u32,f64,f64,f64,f64
"""pass_arrived""",732,1466,3.0,18.2,20.6,11.7
"""ball_snap""",958,1916,6.0,66.3,67.1,1.2
"""pass_outcome_caught""",842,1684,6.0,16.0,18.2,12.1
"""run""",142,284,13.4,35.8,32.5,-10.2
"""handoff""",887,1776,18.8,39.7,40.7,2.5
"""first_contact""",1578,3156,27.2,9.0,12.5,28.0
"""out_of_bounds""",272,544,33.2,3.7,13.0,71.5
"""tackle""",1497,2994,40.1,1.5,9.2,83.7


In [38]:
# split loss up by scramble, run, pass
(
    results_df.filter(pl.col("dataset_split").is_in(["test", "val"]))
    .group_by(["model_type", "tackle_event"])
    .agg(
        n_frames=pl.len(),
        n_plays=pl.struct(["gameId", "playId"]).n_unique(),
        score=pl.map_groups(
            exprs=["tackle_x", "tackle_y", "tackle_x_pred", "tackle_y_pred"],
            function=lambda list_of_series: calculate_mse(*list_of_series),
            # function=lambda list_of_series: calculate_mae(*list_of_series),
            returns_scalar=True,
        ).round(1),
    )
    .sort(["tackle_event", "model_type"])
    .pivot(values="score", columns=["model_type"], index=["tackle_event", "n_plays", "n_frames"])
    .with_columns(perf_perc_diff=((pl.col("zoo") - pl.col("transformer")) * 100 / pl.col("zoo")).round(1))
)

tackle_event,n_plays,n_frames,transformer,zoo,perf_perc_diff
str,u32,u32,f64,f64,f64
"""fumble""",27,1886,35.8,36.0,0.6
"""out_of_bounds""",548,40944,54.8,67.9,19.3
"""qb_slide""",37,2790,22.8,29.9,23.7
"""tackle""",3015,262708,22.5,27.3,17.6
"""touchdown""",116,10296,131.5,164.1,19.9


In [39]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


def animate_play(
    tracking_df: pl.DataFrame,
    play_df: pl.DataFrame,
    results_df: pl.DataFrame,
    gameId: int,
    playId: int,
    mirrored: bool = False,
):
    """
    Animate a play from the tracking data and the results of the models.
    """
    mvmt_df = (
        tracking_df.filter(
            (pl.col("gameId") == gameId) & (pl.col("playId") == playId) & (pl.col("mirrored") == mirrored)
        )
        .to_pandas()
        .round(2)
    )
    play_df = play_df.filter((pl.col("gameId") == gameId) & (pl.col("playId") == playId)).to_pandas()
    model_results_df = results_df.filter(
        (pl.col("gameId") == gameId) & (pl.col("playId") == playId) & (pl.col("mirrored") == mirrored)
    ).to_pandas()
    assert len(mvmt_df) > 0
    assert len(play_df) > 0
    assert len(model_results_df) > 0

    mvmt_df["side"] = mvmt_df["side"].replace({1: "OFF", -1: "DEF"})
    # display(mvmt_df.sample(3), play_df, model_results_df)

    # get some info
    distToGoal = play_df["distanceToGoal"].values[0]
    down = play_df["down"].values[0]
    yards_to_go = play_df["yardsToGo"].values[0]
    play_description = play_df["playDescription"].values[0]
    ballCarrierName = model_results_df["ballCarrierName"].values[0]
    off_club = mvmt_df.loc[mvmt_df["side"] == "OFF", "club"].values[0]
    def_club = mvmt_df.loc[mvmt_df["side"] == "DEF", "club"].values[0]
    dataset_split = model_results_df["dataset_split"].values[0]

    mvmt_y_min = mvmt_df["x"].min()
    mvmt_y_max = mvmt_df["x"].max()

    tkl_x, tkl_y = model_results_df[["tackle_x", "tackle_y"]].values[0]
    # Get prediction data for animation
    tkl_mvmt_df = model_results_df.sort_values("frameId")[["model_type", "frameId", "tackle_x_pred", "tackle_y_pred"]]
    tkl_mvmt_df = tkl_mvmt_df.rename(columns={"tackle_x_pred": "x", "tackle_y_pred": "y"})
    tkl_mvmt_df["displayName"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["nflId"] = tkl_mvmt_df["model_type"].map({"zoo": -10, "transformer": -20})
    tkl_mvmt_df["club"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["side"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["symbol"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["size"] = 1
    tkl_mvmt_df["jerseyNumber"] = ""
    tkl_mvmt_df["is_ball_carrier"] = -1

    # set some things
    # colors_df = pl.read_csv("../data/team_colors.csv")
    # club_color_map = dict(colors_df.select(["club", "secondaryCol"]).rows())
    mvmt_df["size"] = 2
    mvmt_df["text_color"] = "black"

    # Different symbols for different positions
    mvmt_df["symbol"] = "player"
    mvmt_df.loc[mvmt_df["is_ball_carrier"] == 1, "symbol"] = "ball_carrier"
    mvmt_df.loc[mvmt_df["is_ball_carrier"] == 1, "size"] = 3
    # mvnt.loc[mvnt["side"] == "BALL", "symbol"] = "diamond_0"
    symbol_map = {"player": "circle", "ball_carrier": "hexagon", "zoo": "x", "transformer": "x"}

    # operations to do on both tackle and mvmt data
    mvmt_df = pd.concat([mvmt_df, tkl_mvmt_df]).sort_values(["frameId", "nflId"])
    mvmt_df["frameId_grp"] = mvmt_df["frameId"]
    mvmt_df = mvmt_df.groupby("frameId_grp").bfill()

    # display(mvmt_df.head(5))
    mvmt_df["frameId_event"] = mvmt_df["frameId"].astype(str) + mvmt_df["event"].fillna("").astype(str).apply(
        lambda x: f"_{x}" if len(x) > 0 else x
    )

    # Data to display on hover
    hover_data = {
        "displayName": True,
        "club": False,
        "side": False,
        "jerseyNumber": False,
        "is_ball_carrier": True,
        "symbol": False,
        "frameId": False,
        "x": True,
        "y": True,
        "vx": True,
        "vy": True,
        "size": False,
    }

    X_LEFT = 0
    X_MIDDLE = 160 / 6.0
    X_RIGHT = 160 / 3.0
    Y_MIN = 0
    Y_MAX = 120
    Y_MIDDLE = 60
    fig = px.scatter(
        data_frame=mvmt_df,
        x="y",
        y="x",
        animation_frame="frameId_event",
        animation_group="nflId",
        hover_name="displayName",
        hover_data=hover_data,
        text="jerseyNumber",
        width=1000,
        height=900,
        # range_x=[-160 / 6.0, 160 / 6.0],
        range_x=[X_LEFT - 2, X_RIGHT + 2],
        size="size",
        size_max=15,
        color="side",
        color_discrete_map={"OFF": "#39FF14", "DEF": "#FF69B4", "zoo": "blue", "transformer": "gold"},
        opacity=0.9,
        symbol="symbol",
        symbol_map=symbol_map,
    )

    # Add marker for tackle location
    fig.add_trace(
        go.Scatter(
            x=[tkl_y],
            y=[tkl_x],
            mode="markers",
            marker=dict(color="green", size=15, symbol="x"),
            hoverinfo="none",
            showlegend=False,
            opacity=0.8,
        )
    )

    # Add line of scrimmage
    los = Y_MAX - 10 - distToGoal
    fig.add_shape(
        type="line",
        x0=X_LEFT,
        y0=los,
        x1=X_RIGHT,
        y1=los,
        line=dict(color="rgba(137, 207, 240, 0.2)", width=3, dash="dash"),
    )
    # Add yards to go line
    fig.add_shape(
        type="line",
        x0=X_LEFT,
        y0=los + yards_to_go,
        x1=X_RIGHT,
        y1=los + yards_to_go,
        line=dict(color="rgba(255, 255, 0, 0.2)", width=3, dash="dash"),
    )
    # Add border to the field
    fig.add_shape(
        type="rect", x0=X_LEFT, y0=Y_MIN, x1=X_RIGHT, y1=Y_MAX, line=dict(color="rgba(255, 255, 255, 0.5)", width=10)
    )
    # endzone
    fig.add_shape(
        type="rect",
        x0=X_LEFT,
        y0=Y_MAX - 10,
        x1=X_RIGHT,
        y1=Y_MAX,
        line=dict(color="#39FF14", width=6),
        opacity=0.4,
    )
    # Add the path traces to the figure first to place them in the background
    # for trace in path_traces:
    #     fig.add_trace(trace)

    # set play speed
    frame_duration = 100
    for button in fig.layout.updatemenus[0].buttons:
        button["args"][1]["frame"]["duration"] = frame_duration
    # set aspect ratio
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    # background color
    fig.update_layout(paper_bgcolor="#333333", plot_bgcolor="#363636", font_color="white", font_size=14)
    # turn off axis
    fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)
    # grid line thickness
    fig.update_yaxes(
        showgrid=True,
        gridwidth=3,
        gridcolor="rgba(237, 234, 222, 0.1)",
        linewidth=0,
        linecolor="rgba(0, 0, 0, 0.01)",
        mirror=True,
        showticklabels=False,
    )
    # set y axis range
    fig.update_yaxes(range=[mvmt_y_min, mvmt_y_max])
    # set yaxes ticks to 10 yards
    # fig.update_yaxes(tick0=0, dtick=10)
    # text size
    fig.update_layout(uniformtext_minsize=2, uniformtext_mode="hide")
    # hide legend
    fig.update_layout(showlegend=False)
    # text color of jersey numbers
    fig.update_traces(textfont=dict(family="Tahoma", size=12, color=mvmt_df["text_color"]))
    fig.update_traces(marker_line_width=0)

    # hide x and y labels
    fig.update_xaxes(title_text="")
    fig.update_yaxes(title_text="")

    # add hash marks, yard markers, etc
    for y_loc in range(Y_MIN + 10, Y_MAX - 10 + 1, 1):
        if y_loc % 10 == 0:
            ydln = y_loc - 10 if y_loc <= 60 else 110 - y_loc
            ydln_txt = str(ydln) if ydln != 0 else "E Z"
            fig.add_shape(
                type="line", x0=X_LEFT, y0=y_loc, x1=X_RIGHT, y1=y_loc, line=dict(color="white", width=2), opacity=0.05
            )
            fig.add_annotation(
                x=X_LEFT + 4,
                y=y_loc,
                text=ydln_txt,
                showarrow=False,
                font=dict(color="white", size=60),
                textangle=90,
                opacity=0.05,
            )
            fig.add_annotation(
                x=X_RIGHT - 4,
                y=y_loc,
                text=ydln_txt,
                showarrow=False,
                font=dict(color="white", size=60),
                textangle=270,
                opacity=0.05,
            )
        elif y_loc % 5 == 0:
            fig.add_shape(
                type="line", x0=X_LEFT, y0=y_loc, x1=X_RIGHT, y1=y_loc, line=dict(color="white", width=1), opacity=0.05
            )
        else:
            fig.add_shape(
                type="rect",
                x0=X_MIDDLE - 9.5,
                y0=y_loc,
                x1=X_MIDDLE - 8.5,
                y1=y_loc,
                line=dict(color="white", width=3),
                opacity=0.05,
            )
            fig.add_shape(
                type="rect",
                x0=X_MIDDLE + 9.5,
                y0=y_loc,
                x1=X_MIDDLE + 8.5,
                y1=y_loc,
                line=dict(color="white", width=3),
                opacity=0.05,
            )

    fig.add_annotation(
        x=X_MIDDLE, y=Y_MAX - 5, text=off_club, showarrow=False, font=dict(color="white", size=90), opacity=0.2
    )
    fig.add_annotation(
        x=X_MIDDLE, y=Y_MIDDLE + 2, text="SŪMER", showarrow=False, font=dict(color="white", size=50), opacity=0.05
    )
    fig.add_annotation(
        x=X_MIDDLE, y=Y_MIDDLE - 2, text="SPORTS", showarrow=False, font=dict(color="white", size=50), opacity=0.05
    )

    # Add play description
    if play_description is not None:
        # make list of 100 character slices
        play_desc_list = [play_description[i : i + 100] for i in range(0, len(play_description), 100)]
        for i, play_desc_txt in enumerate(play_desc_list):
            text_y_loc = mvmt_y_min - 5 - 1 * i
            fig.add_annotation(
                x=X_MIDDLE,
                y=text_y_loc,
                text=play_desc_txt,
                showarrow=False,
                font=dict(color="white", size=14),
                opacity=0.8,
            )

    # set title
    # offense = play_info["offense"].values[0].upper()
    # defense = play_info["defense"].values[0].upper()
    # down = int(play_info["down"].values[0])
    # yards_to_go = int(play_info["yards_to_go"].values[0])
    # quarter = int(play_info["quarter"].values[0])
    # game_clock = play_info["game_clock"].values[0]
    fig.update_layout(
        title=f"{dataset_split} | {ballCarrierName} | {gameId} {playId} | {off_club} vs {def_club} | Down: {down} | YTG: {yards_to_go} | DTG: {distToGoal}",
        font_size=12,
        title_x=0.5,
        title_y=0.98,
    )

    return fig

In [40]:
gid, pid = (
    results_df.filter(pl.col("dataset_split").is_in(["test"])).select(["gameId", "playId"]).sample(1).to_numpy()[0]
)

# gid, pid = 2022100206, 988 # austin ekeler reverse TD
# gid, pid = 2022092505, 2441 # lamar scramble with a broken tackle # zoo model goes crazy near the end
# gid, pid = 2022102305, 380 # blocker causes prediction to jump
# gid, pid = 2022092501, 2398 # huge hole opens up and prediction jumps # this one is good frames 50-60. Zoo doesnt seem to generalize well in open field situations, even transformer struggles much later
# gid, pid = 2022100909, 1582 # derrik henry dead run -> huge run
# gid, pid = 2022103004, 2631 # Dameon Pierce weaving run
# gid, pid = 2022103002, 3044 # Jet Sweep shows poor generalization from Zoo model
# gid, pid = 2022102307, 641 # Breece Hall huge TD with Transformer model picking up the hole opening (frames 35-40) much more than Zoo model
# gid, pid = 2022091109, 187 # James Rob. big run to left. Sumer model picks up earlier it is going to be a big run
gid, pid = 2022091805, 3127  # long CMC run, frames 40-47 huge difference in prediction


#### OOB Test Set ####
# gid, pid = 2022092200, 1477
# gid, pid = 2022100203, 3117
# gid, pid = 2022100212, 3385
# gid, pid = 2022102307, 1976

print(f"gid, pid = {gid}, {pid}")
animate_play(tracking_df, play_df, results_df, gid, pid)

gid, pid = 2022091805, 3127
